In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os

import datetime

TRAIN_DATASET = 'kiosk_weather_by_day.csv'
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [2]:
from os import getenv
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

status, msg = ds.auth_login_user('danny.diaz@utexas.edu', 'JcXCnQ7Wx2')
if not status:
    print(msg)
else:
    print('You are logged in: ',status)


You are logged in:  True


In [ ]:
# ds.get_info()
# ds.lookup_job_status()
# ds.lookup_job_status_name(job_name)
# ds.delete_job(job_name)
# ds.stop_job(job_name) 


In [ ]:
#ds.upload_dataset(dataset) #must be .csv or .h5


# Data Upload
**Read dataset and view a file snippet**

In [3]:
df = pd.read_csv('./data/kiosk_weather_by_day.csv')
#print(df.dtypes)
df.head(3)

,Date,Month,Year,Weekday,Total Rides,Checkout Kiosk,Return Kiosk,Trip Duration Minutes,Counts,Day Ride Percentage,TempHighF,TempAvgF,DewPointHighF,DewPointAvgF,WindHighMPH,WindAvgMPH,PrecipitationSumInches,PrecipBins,Events
0,2013-12-21,12,2013,5,103,2nd & Congress,2nd & Congress,28,2,0.019417,74,60,67,49,20,4,0.46,0-0.5,"Rain , Thunderstorm"
1,2013-12-21,12,2013,5,103,2nd & Congress,2nd & Congress,33,1,0.009709,74,60,67,49,20,4,0.46,0-0.5,"Rain , Thunderstorm"
2,2013-12-21,12,2013,5,103,2nd & Congress,4th & Congress,5,1,0.009709,74,60,67,49,20,4,0.46,0-0.5,"Rain , Thunderstorm"


**Upload dataset to Darwin**

In [4]:
status, dataset = ds.upload_dataset(f'./data/kiosk_weather_by_day.csv' )
print(status)
print(dataset)

if not status:
    print(dataset)

KeyboardInterrupt: 

# Analyze Data


In [5]:
# The data set cannot be passed in as a relative path, it should be the csv name only
status, analyze_id = ds.analyze_data(TRAIN_DATASET,
                                    job_name='Darwin_analyze_data_job'+'-'+ ts,
                                    artifact_name = 'Darwin_analyze_data_artifact' + 
                                    "-" + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + ts)
else:
    print(analyze_id)


{'status': 'Requested', 'starttime': '2019-04-17T18:50:39.873223', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417185012'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T18:50:39.873223', 'endtime': None, 'percent_complete': 10, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417185012'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T18:50:39.873223', 'endtime': None, 'percent_complete': 10, 'job_type': 'AnalyzeData', 'loss': None, 'generations': None, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': ['Darwin_analyze_data_artifact-20190417185012'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '

In [6]:
ds.lookup_job_status_name(analyze_id['job_name'])

(True,
 {'status': 'Complete',
  'starttime': '2019-04-17T18:50:39.873223',
  'endtime': '2019-04-17T18:51:48.76',
  'percent_complete': 100,
  'job_type': 'AnalyzeData',
  'loss': None,
  'generations': None,
  'dataset_names': ['kiosk_weather_by_day.csv'],
  'artifact_names': ['Darwin_analyze_data_artifact-20190417185012'],
  'model_name': None,
  'job_error': ''})

   # Clean Data

In [7]:
status, job_id = ds.clean_data(dataset_name=TRAIN_DATASET)
if not status:
    print(job_id)
else:
    print('Data has been successfully cleaned!')

Data has been successfully cleaned!


# Create and Train Model

To build unsupervised models, which cluster data and perform anomaly detection, Darwin goes through the following steps:
1. Determines an approximate number of clusters to start with using a single pass with a hierarchical method
2. Iterates on subsets of the data using a Spectral-Net algorithm to determine the ideal number of clusters
3. Proceeds to cluster the data using a Spectral-Net approach

In the cell below, specify the parameters used to create the model:
- model: the name of your model
- max_epochs: the number of epochs to train the model, one epoch indicates one scan of the entire dataset
- n_clusters: the number of clusters, either an integer or 'auto', if left with 'auto', the unsupervised algorithm will compute a number for you

In [10]:
# Build model
model = "model" + "-" + ts
max_epochs = 5
n_clusters = 2
status, job_id = ds.create_model(dataset_names=TRAIN_DATASET,
                                 model_name=model,
                                 max_epochs=max_epochs,
                                 n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)
    if job_id[:3] == '400':
        ds.delete_model(model)
        print(f'Model: {model} deleted!')

{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'l

{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['kiosk_weather_by_day.csv'], 'artifact_names': None, 'model_name': 'model-20190417185012', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T18:58:06.520284', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'l

In [ ]:
ds.wait_for_job(job_id['job_name'])

# Predict


In [ ]:
# Test model
status, artifact = ds.run_model(TRAIN_DATASET, 
                                model, 
                                supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
# Get predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])

In [ ]:
# View prediction
df = pd.read_csv(pred_file['filename'])
df.head()

## Analyze Model
Analyze model provides feature importance ranked by the model. It indicates a general view of which features pose a bigger impact on the model

In [ ]:
status, analyze_id = ds.analyze_model(job_id['model_name'], 
                                      job_name='Darwin_analyze_model_job-' + ts, 
                                      artifact_name='Darwin_analyze_model_artifact-' + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_model_job-' + ts)
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name('Darwin_analyze_model_job-' + ts)